In [1]:
import scripts.stock_plots as stock_plots
import scripts.stock_analysis as stock_ana
import plotly.graph_objects as go

from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import yfinance as yf

import requests
from bs4 import BeautifulSoup

pd.set_option("mode.chained_assignment", None)  # Ignore the warning

In [2]:
url = "https://www.tradingview.com/markets/stocks-usa/market-movers-active/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table")


In [3]:
data = pd.DataFrame(columns=["Ticker", "Vol*Price (M)", "Market cap (M)", "change"])


if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table element on the webpage (you need to inspect the webpage to find the specific HTML tags)
    table = soup.find("table")  # Example: finding the first table on the page

    # Check if the table element was found
    if table:
        # Extract the table rows
        rows = table.find_all("tr")

        # Loop through each row and extract the data
        for row in rows:

            ticker_cell = row.find("td", class_="cell-RLhfr_y4")
            if ticker_cell:
              ticker_symbol = ticker_cell.find("a", class_="tickerName-GrtoTeat").text.strip()
              # print(ticker_symbol)


              value_1_cell = row.find_all("td")[1]
              chg_cell = row.find_all("td")[3]
              value_2_cell = row.find_all("td")[6]

              value_1 = value_1_cell.text.strip().split()
              chg_val = chg_cell.text.strip()
              value_2 = value_2_cell.text.strip().split()


              if len(value_1) == len(value_2):
                def to_million(extract):
                  if extract[1] == "T":
                    return float(extract[0]) * 1e6
                  elif extract[1] == "B":
                    return float(extract[0]) * 1e3
                  else:
                    return float(extract[0])

                vol_price = to_million(value_1)
                market_cap = to_million(value_2)


                if chg_val[0] == "+":
                  chg = float(chg_val[1:-1])
                else:
                  chg = -float(chg_val[1:-1])

                if market_cap < 2000:
                  pass
                else:
                  # print(vol_price, market_cap, relative_active)
                  data.loc[len(data.index)] = [ticker_symbol, vol_price, market_cap, chg]


                  # data = data.append({"Ticker": ticker_symbol, "Vol*Price (M)": vol_price, "Market cap (M)": market_cap, "relative_active": relative_active}, ignore_index=True)
                  # data = pd.concat([ticker_symbol, vol_price, market_cap, relative_active], ignore_index=True)

    else:
        print("Table element not found on the webpage.")
else:
    print("Failed to retrieve webpage. Status code:", response.status_code)

In [4]:
data["rel_active"] = data["Vol*Price (M)"] / data["Market cap (M)"] * (abs(data["change"]))
data["rel_active(sqrt)"] = data["Vol*Price (M)"] / np.sqrt(data["Market cap (M)"]) * (abs(data["change"]))
data["rel_active(log)"] = data["Vol*Price (M)"] / np.log(data["Market cap (M)"]) * (abs(data["change"]))


data["rel_active() rank"] = data["rel_active"].rank(ascending=False)
data["rel_active(sqrt) rank"] = data["rel_active(sqrt)"].rank(ascending=False)
data["rel_active(log) rank"] = data["rel_active(log)"].rank(ascending=False)

data["rel_rank"] = data["rel_active() rank"]**2 + data["rel_active(sqrt) rank"]**2 + data["rel_active(log) rank"]**2
data["rel_active(rel_rank) rank"] = data["rel_rank"].rank()


In [7]:
data.sort_values(by="rel_active(rel_rank) rank").head(10)

,Ticker,Vol*Price (M),Market cap (M),change,rel_active,rel_active(sqrt),rel_active(log),rel_active() rank,rel_active(sqrt) rank,rel_active(log) rank,rel_rank,rel_active(rel_rank) rank
4,SMCI,6952.000,57163.0,10.60,1.289142,308.218088,6727.539998,2.0,1.0,1.0,6.0,1.0
92,ITCI,589.327,7729.0,23.29,1.775835,156.122024,1533.098677,1.0,2.0,6.0,41.0,2.0
1,TSLA,15240.000,500362.0,-2.71,0.082541,58.386454,3147.155821,6.0,4.0,3.0,61.0,3.0
16,MSTR,2168.000,21279.0,-6.13,0.624552,91.105405,1333.588086,3.0,3.0,8.0,82.0,4.0
7,UNH,5541.000,432286.0,5.22,0.066909,43.991935,2228.895017,8.0,6.0,4.0,116.0,5.0
39,LYV,927.995,21282.0,-7.58,0.330524,48.217937,705.847130,4.0,5.0,10.0,141.0,6.0
3,AMD,9040.000,264117.0,1.96,0.067085,34.476750,1419.271925,7.0,8.0,7.0,162.0,7.0
9,COIN,3605.000,53017.0,-2.05,0.139394,32.096048,679.352829,5.0,9.0,11.0,227.0,8.0
10,BAC,3314.000,273024.0,-3.53,0.042848,22.388593,934.579022,11.0,10.0,9.0,302.0,9.0
0,NVDA,32381.000,2185000.0,1.64,0.024304,35.925960,3638.033856,17.0,7.0,2.0,342.0,10.0


In [12]:
def download_data(stocks):
    stocks_str = " ".join(stocks)
    return yf.Tickers(stocks_str)

In [14]:
selected_tickers = data.sort_values(by="rel_active(rel_rank) rank")["Ticker"].head(10).values

stock_data = download_data(selected_tickers)



for ticker in selected_tickers:
    stock_plot = stock_plots.PlotInfo(stock_data.tickers[ticker], ticker, "150d")
    candle = stock_plot.generate_candle_plot(p2p_order=4)
    candle.show()